<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/govLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Sample ISDA agreement data
data = {
    "file_name": ["file1", "file2", "file3", "file4"],
    "category": ["ISDA", "Amendment", "ISDA", "Amendment"],
    "date": ["2023-05-01", "2023-06-15", "2023-04-01", "2023-07-20"],
    "date_of_original_agreement": [None, "2023-05-01", None, "2023-04-01"],
    "summary": [
        "Initial agreement signed.",
        "Governing Law changed to NY law.",
        "Another agreement signed.",
        "Governing Law changed to UK law."
    ],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Convert dates to datetime for easier processing
df['date'] = pd.to_datetime(df['date'])
df['date_of_original_agreement'] = pd.to_datetime(df['date_of_original_agreement'], errors='coerce')

# Function to extract governing law from summary
def extract_governing_law(summary):
    if "Governing Law changed to" in summary:
        return summary.split("Governing Law changed to")[1].strip().rstrip(".")
    return None

# Add a new column for the governing law update
df['new_governing_law'] = df['summary'].apply(extract_governing_law)

# Identify and update the original agreement
def update_governing_law(row, df):
    if pd.notna(row['date_of_original_agreement']):
        # Find the original agreement
        original_agreement = df[
            (df['date'] == row['date_of_original_agreement']) &
            (df['category'] == "ISDA")
        ]
        if not original_agreement.empty:
            original_index = original_agreement.index[0]
            # Update the original agreement's governing law
            df.at[original_index, 'final_gov_law'] = row['new_governing_law']

# Initialize the new column
df['final_gov_law'] = None

# Process each row with 'Governing Law changed'
df.apply(lambda row: update_governing_law(row, df), axis=1)

# Fill NaN in final_gov_law with placeholders
df['final_gov_law'] = df['final_gov_law'].fillna("No Change")

# Display the updated DataFrame
print(df)
